# Map Model
Contains codes for a simple area-preseving map model of the more complex gyrokinetic Poincare map

In [79]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

In [118]:
# %% Define and test the simple map model

# Zonal flow curvature
curv = 0.75*2
# Envelope scale
jscale = 0.1
# Perturbation amplitude
eps = -0.0025

# Envelope function and its derivative
def env_func(j):
    return np.exp(-0.5 * j**2 / jscale**2)
def denv_func(j):
    return -j / jscale**2 * env_func(j)

# Define the map function
def map_advance(j, theta):
    # Generating function is - A(J) * cos(n * theta)

    # Compute the new action; implict solve using Newton iteration
    j2_guess = j + env_func(j) * np.sin(39*theta) * eps * 39
    j2 = scipy.optimize.newton(
        lambda j2: j2 - j - env_func(j2) * np.sin(39*theta) * eps * 39,
        j2_guess,
        fprime= lambda j2: 1 - denv_func(j2) * np.sin(39*theta) * eps * 39
    )
    #j2 = j + np.sin(39*theta) * eps

    theta2 = theta + 0.03 + 0.75 * 0.5 * curv * j2**2 + denv_func(j2) * np.cos(39*theta) * eps

    return j2, theta2

# Number of initial particles and number of steps to advance
num_p = 128
num_steps = 1024



# Initialize particles
j = np.zeros((num_p, num_steps))
theta = np.zeros((num_p, num_steps))
j[:, 0] = np.linspace(-0.15, 0.15, num_p)
theta[:, 0] = np.linspace(0, 2*np.pi, num_p, endpoint=False)

# Advance particles through the map
for step in range(1, num_steps):
    for p in range(num_p):
        j[p, step], theta[p, step] = map_advance(j[p, step-1], theta[p, step-1])




In [117]:
# %% Plot the results

%matplotlib qt

j0 = np.zeros((num_p, num_steps))
j0[:, :] = (j[:, 0])[:, np.newaxis]

plt.figure()
plt.scatter(j[:, :], np.mod(theta[:, :],2*np.pi/39), s=1, c=j0, lw=0)